In [ ]:
# %%
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [ ]:
model = AutoModelForMaskedLM.from_pretrained(
    "ibm-research/MoLFormer-XL-both-10pct", trust_remote_code=True, dtype="auto"
)
tokenizer = AutoTokenizer.from_pretrained(
    "ibm/MoLFormer-XL-both-10pct", trust_remote_code=True
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenization_molformer_fast.py: 0.00B [00:00, ?B/s]

tokenization_molformer.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ibm/MoLFormer-XL-both-10pct:
- tokenization_molformer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/ibm/MoLFormer-XL-both-10pct:
- tokenization_molformer_fast.py
- tokenization_molformer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
compounds = pd.read_csv(
    "/Users/lucas-diedrich/Documents/Programming/nucleate-hackathon/notebooks/cas-to-smiles/unique_compounds.tsv",
    sep="\t",
    index_col=0,
)
compounds

,Ranking,Catalog Number,Score,Compound name,Treatment concentration,CAS Number,CID,CAS,smiles
0,1,HY_107369,0.367344,4-Butylresorcinol,10 μM,18979-61-8,205912.0,18979-61-8,CCCCC1=C(C=C(C=C1)O)O
9,3,HY_19747,0.345520,HPOB,10 μM,1429651-50-2,71532921.0,1429651-50-2,C1=CC=C(C=C1)N(CCO)C(=O)CC2=CC=C(C=C2)C(=O)NO
30,8,HY_11009,0.331873,CGP60474,10 μM,164658-13-3,644215.0,164658-13-3,C1=CC(=CC(=C1)Cl)NC2=NC=CC(=N2)C3=CC(=NC=C3)NCCCO
31,9,HY_12798C,0.329573,AR-13324 M1 metabolite,10 μM,2309668-15-1,134128281.0,2309668-15-1,C1=CC(=CC=C1CO)[C@@H](CN)C(=O)NC2=CC3=C(C=C2)C...
62,15,HY_122641B,0.311769,Deltasonamide 2 (TFA),10 μM,2235358-74-2,138454775.0,2235358-74-2,CNC1=NC=CC(=N1)CN(C2CCC(CC2)N)S(=O)(=O)C3=CC=C...
...,...,...,...,...,...,...,...,...,...
49993,12336,HY_122614,-0.250347,S29434,10 μM,874484-20-5,46861897.0,874484-20-5,COC1=NC2=C(C=C1)N3CC4=C(C3=C2CCNC(=O)C5=CC=CO5...
50000,12341,HY_13504,-0.253410,PIK-293,10 μM,900185-01-5,53245636.0,900185-01-5,CC1=C2C(=CC=C1)N=C(N(C2=O)C3=CC=CC=C3C)CN4C5=N...
50018,12345,HY_13994,-0.255190,Mps1-IN-2,10 μM,1228817-38-6,44968267.0,1228817-38-6,CCOC1=C(C=CC(=C1)N2CCC(CC2)O)NC3=NC=C4C(=N3)N(...
50023,12348,HY_135954A,-0.267305,PDK4-IN-1 (hydrochloride),10 μM,2310262-11-2,146026197.0,2310262-11-2,C1CNCCC1N2C=C(C=N2)C3=C4C(=CC=C3)C(=O)C5=CC=CC...


In [ ]:
# %%
model = AutoModelForMaskedLM.from_pretrained(
    "ibm-research/MoLFormer-XL-both-10pct", trust_remote_code=True, dtype="auto"
)
tokenizer = AutoTokenizer.from_pretrained(
    "ibm/MoLFormer-XL-both-10pct", trust_remote_code=True
)

# %%
compounds = pd.read_csv(
    "/Users/lucas-diedrich/Documents/Programming/nucleate-hackathon/notebooks/cas-to-smiles/unique_compounds.tsv",
    sep="\t",
    index_col=0,
)
compounds

# %%
# Filter out invalid SMILES (NaN, None, empty strings)
smiles = compounds["smiles"].dropna().astype(str).tolist()
smiles = [s for s in smiles if s and s.strip() and s != "nan"]
print(f"Number of valid SMILES: {len(smiles)}")
smiles[:5]  # Show first 5

# %%
# Process in batches to avoid memory issues
batch_size = 64
all_embeddings = []

for i in range(0, len(smiles), batch_size):
    batch = smiles[i : i + batch_size]
    print(
        f"Processing batch {i // batch_size + 1}/{(len(smiles) - 1) // batch_size + 1} ({len(batch)} compounds)"
    )

    # Tokenize and get embeddings
    inputs = tokenizer(
        batch, padding=True, return_tensors="pt", truncation=True, max_length=512
    )

    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)

    # Extract embeddings from the last hidden state (mean pooling)
    # Shape: (batch_size, sequence_length, hidden_size)
    hidden_states = outputs.hidden_states[-1]

    # Mean pooling over sequence length
    # Shape: (batch_size, hidden_size)
    embeddings = hidden_states.mean(dim=1)

    all_embeddings.append(embeddings)

# Concatenate all batch embeddings
all_embeddings = torch.cat(all_embeddings, dim=0)
print(f"\nFinal embeddings shape: {all_embeddings.shape}")
all_embeddings

Number of valid SMILES: 2248
Processing batch 1/36 (64 compounds)
Processing batch 2/36 (64 compounds)
Processing batch 2/36 (64 compounds)
Processing batch 3/36 (64 compounds)
Processing batch 3/36 (64 compounds)
Processing batch 4/36 (64 compounds)
Processing batch 4/36 (64 compounds)
Processing batch 5/36 (64 compounds)
Processing batch 5/36 (64 compounds)
Processing batch 6/36 (64 compounds)
Processing batch 6/36 (64 compounds)
Processing batch 7/36 (64 compounds)
Processing batch 7/36 (64 compounds)
Processing batch 8/36 (64 compounds)
Processing batch 8/36 (64 compounds)
Processing batch 9/36 (64 compounds)
Processing batch 9/36 (64 compounds)
Processing batch 10/36 (64 compounds)
Processing batch 10/36 (64 compounds)
Processing batch 11/36 (64 compounds)
Processing batch 11/36 (64 compounds)
Processing batch 12/36 (64 compounds)
Processing batch 12/36 (64 compounds)
Processing batch 13/36 (64 compounds)
Processing batch 13/36 (64 compounds)
Processing batch 14/36 (64 compounds)


tensor([[ 0.9853, -0.4144, -0.3637,  ...,  0.1250,  0.2976,  0.7357],
        [-0.2872, -0.7226, -0.7181,  ...,  0.0578, -0.5348,  0.0419],
        [-0.1828, -0.5759, -0.5660,  ..., -0.0396, -1.1061, -0.1290],
        ...,
        [ 0.7290,  0.0519,  0.2238,  ..., -0.1652, -0.7321, -0.3948],
        [ 0.0993, -0.0477,  0.2584,  ..., -0.0532, -0.3529, -0.2720],
        [ 1.1721, -0.4888, -0.3700,  ...,  0.3060, -0.2637, -0.3691]])

In [ ]:
all_embeddings_numpy = all_embeddings.detach().numpy()
all_embeddings_numpy

array([[ 0.98525286, -0.4144381 , -0.36374164, ...,  0.12501554,
         0.2975823 ,  0.7357472 ],
       [-0.287197  , -0.7225522 , -0.7181257 , ...,  0.0577708 ,
        -0.53479666,  0.04189637],
       [-0.18279272, -0.5759117 , -0.56602603, ..., -0.03956044,
        -1.1060615 , -0.12903222],
       ...,
       [ 0.7290484 ,  0.05186475,  0.22384894, ..., -0.16515757,
        -0.7320868 , -0.39475122],
       [ 0.09933088, -0.04774359,  0.25837204, ..., -0.05320119,
        -0.35293302, -0.2719858 ],
       [ 1.1721483 , -0.48880216, -0.37001225, ...,  0.30596623,
        -0.2636613 , -0.36910692]], shape=(2248, 768), dtype=float32)

In [ ]:
df = pd.DataFrame(all_embeddings_numpy, index=smiles)
df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
CCCCC1=C(C=C(C=C1)O)O,0.985253,-0.414438,-0.363742,0.622728,-1.328449,-0.324084,-0.641040,-0.075589,-0.165918,-0.746966,...,0.767878,0.810009,-0.503663,0.985284,-0.176481,-0.664044,-0.132383,0.125016,0.297582,0.735747
C1=CC=C(C=C1)N(CCO)C(=O)CC2=CC=C(C=C2)C(=O)NO,-0.287197,-0.722552,-0.718126,-0.236562,-1.115782,-0.531389,-1.278204,0.622446,0.120421,-0.455159,...,0.422765,0.519235,-0.135786,-0.071139,-0.080451,-0.388809,-1.724904,0.057771,-0.534797,0.041896
C1=CC(=CC(=C1)Cl)NC2=NC=CC(=N2)C3=CC(=NC=C3)NCCCO,-0.182793,-0.575912,-0.566026,0.463177,-0.747918,-0.133567,-0.460694,-0.023779,-0.798178,-0.630170,...,0.553366,0.201456,-0.415808,0.066159,-0.294800,-0.380439,-1.920309,-0.039560,-1.106061,-0.129032
C1=CC(=CC=C1CO)[C@@H](CN)C(=O)NC2=CC3=C(C=C2)C=NC=C3,-0.150467,-0.206697,-0.575187,0.074890,-1.097888,-0.845826,-0.927723,0.283547,0.041936,-0.621553,...,0.666575,0.389585,0.147568,-0.148481,0.153590,-0.205970,-1.729239,-0.107210,-0.782025,0.112719
CNC1=NC=CC(=N1)CN(C2CCC(CC2)N)S(=O)(=O)C3=CC=C(C=C3)S(=O)(=O)N(CC4=CC=C(C=C4)Cl)C5CCCC5.C(=O)(C(F)(F)F)O,0.306828,-0.736980,-0.447893,0.494913,-0.529202,-0.574852,-0.641105,0.627730,0.519676,-0.460715,...,0.494807,0.747837,0.614527,0.145446,-0.491479,-0.256579,-1.946391,-0.606912,-0.096466,0.018026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
COC1=NC(=NC=C1)N2CCC3(CCCN(C3=O)CC4=CNC5=CC=CC=C54)CC2,0.594084,0.241800,0.061521,0.398166,-0.523359,0.018218,-0.518841,0.376587,-0.176870,0.063586,...,-0.374233,0.462094,-0.076321,0.466713,0.331940,-0.455478,-2.999456,-0.281490,-0.756438,-0.443556
COC1=NC2=C(C=C1)N3CC4=C(C3=C2CCNC(=O)C5=CC=CO5)N=CC=C4,0.683878,0.062158,0.541062,0.401861,-0.102901,0.076197,-0.537647,0.286872,-0.445591,-0.513493,...,-0.146895,0.425641,0.079641,0.524890,0.517666,-0.573386,-2.358850,-0.352562,-0.681536,-0.531223
CC1=C2C(=CC=C1)N=C(N(C2=O)C3=CC=CC=C3C)CN4C5=NC=NC(=C5C=N4)N,0.729048,0.051865,0.223849,0.365191,-0.668336,-0.111063,-0.396522,0.474470,-0.604657,-0.662982,...,-0.262879,0.575145,-0.234254,0.888520,0.317051,-0.308241,-2.980067,-0.165158,-0.732087,-0.394751
CCOC1=C(C=CC(=C1)N2CCC(CC2)O)NC3=NC=C4C(=N3)N(CCC(=O)N4C)C5CCCC5,0.099331,-0.047744,0.258372,0.093163,-0.322585,0.200407,-0.373826,0.576085,-0.035309,-0.590278,...,-0.327709,0.654967,-0.125509,0.752127,0.156419,-0.438584,-3.248656,-0.053201,-0.352933,-0.271986


In [ ]:
merged_df = pd.read_csv(
    "/Users/lucas-diedrich/Documents/Programming/nucleate-hackathon/notebooks/cas-to-smiles/CAS_to_smiles.tsv",
    sep="\t",
    index_col=0,
)
merged_df

,CID,CAS,smiles
0,2,5080-50-2,CC(=O)OC(CC(=O)O)C[N+](C)(C)C
1,7,2715-68-6,CCN1C=NC2=C(N=CN=C21)N
2,19,303-38-8,C1=CC(=C(C(=C1)O)O)C(=O)O
3,43,103404-90-6,C(CC(=O)O)C(C(=O)O)O
4,47,1460-34-0,CCC(C)C(=O)C(=O)O
...,...,...,...
45831,176541418,474-25-9,C[C@H](CCC(=O)O)[C@H]1CCC2[C@@]1(CC[C@@H]3[C@@...
45832,176542861,13292-46-1,C[C@H]1/C=C(/C=C(/C(=O)NC2=C(C(=C3C(=C2O)C(=C(...
45833,176544248,129-16-8,C1=CC=C2C(=C1)C(=O)OC23C4=CC(=C(C=C4OC5=C=C([C...
45834,176546160,3810-74-0,C[C@@H]1[C@]([C@@H]([C@H](O1)O[C@@H]2[C@H]([C@...


In [ ]:
res = df.merge(merged_df, left_index=True, right_on="smiles").set_index(
    ["CID", "CAS", "smiles"]
)
res.to_csv("./molformer-embeddings.tsv", sep="\t")

In [ ]:
res.shape

(2379, 768)